In [21]:
def create_files_from_lines(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    for i, line in enumerate(lines):
        with open(f'./Nivo3A_parsed/new_file_{i}.txt', 'w') as new_file:
            new_file.write(line)

create_files_from_lines('Nivo3A.txt')

In [22]:
import os
import json
import tiktoken
import openai
import numpy as np
from dotenv import load_dotenv
from openai.embeddings_utils import cosine_similarity
from tenacity import retry, wait_random_exponential, stop_after_attempt

# Load environment variables
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = "https://domzisopenai.openai.azure.com/"
openai.api_key = "3673181a896c4997baba06bc82db5e6f"

# Define embedding model and encoding
EMBEDDING_MODEL = 'TextEmbeddingAda002'
EMBEDDING_ENCODING = 'cl100k_base'
EMBEDDING_CHUNK_SIZE = 8000
COMPLETION_MODEL = 'TextDavinci003'

# initialize tiktoken for encoding text
encoding = tiktoken.get_encoding(EMBEDDING_ENCODING)

In [23]:
# list all files in the samples directory
samples_dir = os.path.join(os.getcwd(), "./QuetionsParsed/")
sample_files = os.listdir(samples_dir)

# read each file and remove and newlines (better for embeddings later)
documents = []
for file in sample_files:
    with open(os.path.join(samples_dir, file), "r") as f:
        content = f.read()
        content = content.replace("\n", " ")
        content = content.replace("  ", " ")
        documents.append(content)

# print some stats about the documents
print(f"Loaded {len(documents)} documents")
for doc in documents:
    num_tokens = len(encoding.encode(doc))
    print(f"Content: {doc[:80]}... \n---> Tokens: {num_tokens}\n")

Loaded 37 documents
Content: Kakšen je učinek davčne olajšave? ... 
---> Tokens: 18

Content: Kakšne so možnosti dodatnih vplačil v zavarovanje? ... 
---> Tokens: 22

Content: Kje lahko preverim podatke o izvedeni investiciji? ... 
---> Tokens: 19

Content: Kakšne so možnosti dodatnih vplačil v zavarovanje? ... 
---> Tokens: 22

Content: Kakšne so možnosti dodatnih vplačil v zavarovanje? ... 
---> Tokens: 22

Content: Kakšne so možnosti predčasnega izplačila? ... 
---> Tokens: 18

Content: Kaj storiti v primeru smrti zavarovanca? ... 
---> Tokens: 16

Content: Kako preverim stanje na skladih? ... 
---> Tokens: 13

Content: Kakšne so možnosti dodatnih vplačil v zavarovanje? ... 
---> Tokens: 22

Content: Kaj so koristi kasko zavarovanja? ... 
---> Tokens: 14

Content: Kaj krije obvezno avtomobilsko zavarovanje?... 
---> Tokens: 18

Content: Na kaj je vezano zavarovanje mladi voznik? ... 
---> Tokens: 17

Content: Kako lahko preverim vrednosti enot premoženja (VEP) skladov? ... 
---> Tok

In [25]:
#test code
def get_embedding(text):
    return openai.Embedding.create(input=text, engine=EMBEDDING_MODEL)["data"][0]["embedding"]

get_embedding("This is a test string")

[-0.007274896837770939,
 0.011575490236282349,
 -0.0030399614479392767,
 -0.010446174070239067,
 -0.017990268766880035,
 0.013413911685347557,
 -0.009257766418159008,
 0.0030268297996371984,
 -0.008830989710986614,
 -0.028364218771457672,
 0.009192108176648617,
 0.017556926235556602,
 -0.015048794448375702,
 0.018003400415182114,
 0.0035225471947342157,
 0.010104753077030182,
 0.0204590056091547,
 3.847143489110749e-06,
 0.019040795043110847,
 -0.00591577822342515,
 -0.0007485002279281616,
 0.006628166418522596,
 -0.011536095291376114,
 0.025751033797860146,
 -0.01711045205593109,
 -0.007938042283058167,
 -0.0012286236742511392,
 -0.02062971703708172,
 0.012626017443835735,
 -0.007517831400036812,
 0.01575789973139763,
 -0.01575789973139763,
 -0.003450323361903429,
 -0.021023664623498917,
 -0.002181484131142497,
 -0.0006569895194843411,
 -0.007294594310224056,
 -0.026670245453715324,
 0.020747901871800423,
 -0.01902766339480877,
 0.010859819129109383,
 0.01213358249515295,
 2.303477231

In [26]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text):
    return openai.Embedding.create(input=text, engine=EMBEDDING_MODEL)["data"][0]["embedding"]

# Create embeddings for all docs
embeddings = [get_embedding(doc) for doc in documents]

# print some stats about the embeddings
for e in embeddings:
    print(e)

[-0.004192939028143883, -0.000732768967282027, 0.019025959074497223, -0.037708885967731476, -0.0019417994190007448, -0.0048575615510344505, -0.01778859831392765, -0.016551237553358078, -0.019650764763355255, -0.010842223651707172, 0.010640081018209457, 0.004416521638631821, 0.007326158229261637, 0.025874324142932892, -0.007595682516694069, 0.013243439607322216, 0.04317287728190422, -0.03538117930293083, 0.01581617072224617, -0.02558029815554619, -0.025947831571102142, 0.014860585331916809, -0.04398145154118538, 0.004741176031529903, -0.009218953549861908, 0.0019433308625593781, 0.00473505025729537, -0.009212827309966087, 0.01102599035948515, -3.2948759326245636e-05, -0.003329236526042223, -0.013133179396390915, -0.011620168574154377, -0.009476225823163986, -0.019381241872906685, -0.014983096159994602, 0.0027243385557085276, -0.0020459338556975126, 0.03486663103103638, -0.007773323450237513, 0.02415917068719864, 0.0026447062846273184, -0.006131675560027361, -0.012937162071466446, -0.015

In [ ]:
# create embedding for question
question = "Potrebujem novega izvajalca za popravilo hladilnika oz hladilnega sistema"
qe = get_embedding(question)

# calculate cosine similarity between question and each document
similaries = [cosine_similarity(qe, e) for e in embeddings]

# Get the matching document, in this case we just use argmax of similarities
max_i = np.argmax(similaries)

# print some stats about the similarities
for i, s in enumerate(similaries):
    print(f"Similarity to {sample_files[i]} is {s}")
print(f"Matching document is {sample_files[max_i]}")

Similarity to new_file_5.txt is 0.7672372581929093
Similarity to new_file_7.txt is 0.7750804499606696
Similarity to new_file_23.txt is 0.786096572017221
Similarity to new_file_16.txt is 0.7750804499606696
Similarity to new_file_4.txt is 0.7750804499606696
Similarity to new_file_10.txt is 0.7811793648384414
Similarity to new_file_19.txt is 0.7643814580796757
Similarity to new_file_25.txt is 0.8066957257979449
Similarity to new_file_2.txt is 0.7750804499606696
Similarity to new_file_33.txt is 0.7672965564472282
Similarity to new_file_36.txt is 0.7748856625238284
Similarity to new_file_30.txt is 0.7869527693316755
Similarity to new_file_24.txt is 0.7739953309527128
Similarity to new_file_0.txt is 0.7957973735796081
Similarity to new_file_27.txt is 0.7789617809086704
Similarity to new_file_17.txt is 0.7784632487698419
Similarity to new_file_20.txt is 0.7781018980368536
Similarity to new_file_26.txt is 0.8021109319127696
Similarity to new_file_8.txt is 0.7916501756042968
Similarity to new_f

In [6]:
# pip install redis
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
#from langchain.vectorstores.redis import Redis

In [7]:
#texts = [d for d in documents]


# list all files in the samples directory
samples_dir = os.path.join(os.getcwd(), "./Nivo3_parsed/")
sample_files = os.listdir(samples_dir)

# read each file and remove and newlines (better for embeddings later)
nivo3 = []
for file in sample_files:
    with open(os.path.join(samples_dir, file), "r") as f:
        content = f.read()
        content = content.replace("\n", " ")
        content = content.replace("  ", " ")
        nivo3.append(content)


In [8]:
# list all files in the samples directory
samples_dir = os.path.join(os.getcwd(), "./Nivo3A_parsed/")
sample_files = os.listdir(samples_dir)

# read each file and remove and newlines (better for embeddings later)
nivo3A = []
for file in sample_files:
    with open(os.path.join(samples_dir, file), "r") as f:
        content = f.read()
        content = content.replace("\n", " ")
        content = content.replace("  ", " ")
        nivo3A.append(content)

In [9]:
texts = [d for d in documents]
t_nivo3 = [d for d in nivo3]
t_nivo3A = [d for d in nivo3A]

print(texts)
print(t_nivo3)
print(t_nivo3A)

['Kakšen je učinek davčne olajšave? ', 'Kakšne so možnosti dodatnih vplačil v zavarovanje? ', 'Kje lahko preverim podatke o izvedeni investiciji? ', 'Kakšne so možnosti dodatnih vplačil v zavarovanje? ', 'Kakšne so možnosti dodatnih vplačil v zavarovanje? ', 'Kakšne so možnosti predčasnega izplačila? ', 'Kaj storiti v primeru smrti zavarovanca? ', 'Kako preverim stanje na skladih? ', 'Kakšne so možnosti dodatnih vplačil v zavarovanje? ', 'Kaj so koristi kasko zavarovanja? ', 'Kaj krije obvezno avtomobilsko zavarovanje?', 'Na kaj je vezano zavarovanje mladi voznik? ', 'Kako lahko preverim vrednosti enot premoženja (VEP) skladov? ', 'Kaj je Triglav komplet? ', 'Kaj je varčevalni načrt? ', 'Kdo je zavarovalec na polici? ', 'Koliko časa traja zavarovalna doba? ', 'Kje dobim več informacij o skladih? ', 'Kje lahko dobim dodatne informacije o zavarovanju? ', 'Kje lahko dobim dodatne informacije o zavarovanju? ', 'Kakšne so prednosti varčevalnega načrta? ', 'Kako lahko prekinem polico? (sem z

In [27]:
#texts
len(embeddings[0])

1536

In [11]:
REDIS_INDEX_NAME = 'domtistestindex'
VECTOR_FIELD_IN_REDIS='item_vector'
NUMBER_PRODUCTS_INDEX=1000
CHOSEN_EMB_MODEL = 'TextEmbeddingAda002'
REDIS_ADDR = 'redisdbtriguc3.westeurope.redisenterprise.cache.azure.net'
REDIS_PORT = 10000
REDIS_PASSWORD = 'hB+FWOy6MDlsL21bmaD1plSNwTa6Vd70TnID02P0r+0='


In [12]:
import redis
r = redis.StrictRedis(host='redisdbtriguc3.westeurope.redisenterprise.cache.azure.net',
    port=10000, db=0, password='hB+FWOy6MDlsL21bmaD1plSNwTa6Vd70TnID02P0r+0=', ssl=True)

In [13]:
import redis
from redis.commands.search.field import TagField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

r = redis.StrictRedis(host='redisdbtriguc3.westeurope.redisenterprise.cache.azure.net',
    port=10000, db=0, password='hB+FWOy6MDlsL21bmaD1plSNwTa6Vd70TnID02P0r+0=', ssl=True)

INDEX_NAME = "domzis_index"                              # Vector Index Name
DOC_PREFIX = "doc:"                               # RediSearch Key Prefix for the Index

def create_index(vector_dimensions: int):
    try:
        # check to see if index exists
        r.ft(INDEX_NAME).info()
        print("Index already exists!")
    except:
        # schema
        schema = (
            TagField("tag"),                       # Tag Field Name
            VectorField("vector",                  # Vector Field Name
                "FLAT", {                          # Vector Index Type: FLAT or HNSW
                    "TYPE": "FLOAT32",             # FLOAT32 or FLOAT64
                    "DIM": vector_dimensions,      # Number of Vector Dimensions
                    "DISTANCE_METRIC": "COSINE",   # Vector Search Distance Metric
                }
            ),
        )

        # index Definition
        definition = IndexDefinition(prefix=[DOC_PREFIX], index_type=IndexType.HASH)

        # create Index
        r.ft(INDEX_NAME).create_index(fields=schema, definition=definition)


In [14]:
r.ping()

True

In [28]:
# define vector dimensions
VECTOR_DIMENSIONS = 1536

# create the index
create_index(vector_dimensions=VECTOR_DIMENSIONS)

Index already exists!


### This is a form a tutorial on redis-py : https://redis-py.readthedocs.io/en/stable/examples/search_vector_similarity_examples.html

In [22]:
#documents

In [24]:
# Drop the index
r.ft(INDEX_NAME).dropindex(delete_documents=True)

b'OK'

In [25]:
# Create a new index
create_index(vector_dimensions=VECTOR_DIMENSIONS)

In [26]:
#len(texts)
#len(embeddings)

In [29]:
np_embedding = np.array(embeddings, dtype=np.float32)


for i, embedding in enumerate(np_embedding):
    print(embedding.tobytes())

b'\xecd\x89\xbbK\x17@\xbaT\xdc\x9b<\xa2t\x1a\xbd\xfa\x83\xfe\xba.,\x9f\xbbe\xb9\x91\xbcv\x96\x87\xbc\xa4\xfa\xa0\xbc\x95\xa31\xbc\xbcS.<x\xb8\x90;E\x10\xf0;d\xf6\xd3<4\xe5\xf8\xbb\x03\xfbX<\x0b\xd60=\xdb\xeb\x10\xbd\xea\x90\x81<\xc6\x8d\xd1\xbc\x8c\x90\xd4\xbc\xd0ys<\xe5%4\xbd\xde[\x9b;\x18\x0b\x17\xbc]\xb7\xfe:{(\x9b;f\xf1\x16\xbc[\xa64<v2\n\xb8R/Z\xbb\x8c,W\xbc\x85b>\xbc,B\x1b\xbci\xc5\x9e\xbc\xa9{u\xbc\xd1\x8a2;\x13\x15\x06\xbbP\xd0\x0e=]\xb7\xfe\xbbt\xe9\xc5<\xcfR-;9\xec\xc8\xbbd\xf6S\xbc\x9b\xaa\x81\xbc(%\xcf<8f\xc2<P\x82\r\xbd4\x0c\xb4\xbcu\xd3\xc9;a\xfb\x85<\xc1\x97\xc0<c\x81\x8c\xbb\xe9\xcd\xc3\xbb\xb76\xe2\xbc(\xc1Q\xbc\x94k,<\xd2\x10\xb9\xbb\xff\xa0J\xba}9\xe5;\xcd0$;\xbc\xef\xb0<=\xd1\x8f\xbc\xedN\x8d<\x05\xf6\x1b\xbd\xd1&5\xbc\xb8G\xa1\xbb\x1c\xdal:\xedN\x8d\xbc\x83@\xb5<\x182]=\xecd\t=<5\x8d\xba\xc0\xad\xbc\xbcRV\x95<\xa8T/\xbc\x82\xa42\xbc\xd9\xb3\x8b\xbc\x00d\x88<\x98P~<\xa8\xdf\xf2<N\x87\xca\xbb\xb0\x93\x04\xbca\xd4J\xbc4\x0c4<G\xbd\xb1\xbc\xf4\x18&\xbc}\xae!=\xfd\xcc\x

In [29]:
import numpy as np

np_embedding = np.array(embeddings, dtype=np.float32)

# Write to Redis
pipe = r.pipeline()
ct = 0
for i, embedding in enumerate(np_embedding):
    by_em = embedding.tobytes()
    pipe.hset(f"doc:{i}", mapping = {
        "vector": by_em,
        "content": texts[ct],
        "nivo3": t_nivo3[ct],
        "nivo3A": t_nivo3A[ct],
        "tag": "openai"
    })
    ct += 1
    res = pipe.execute()


In [30]:
# expected output for question: Kako lahko sklenem zavarovanje?

q_em = get_embedding("Rad bi sklenil zavarovanje pri Triglav d.d.")

In [31]:
q_em = np.array(q_em, dtype=np.float32)

In [32]:
q_em

array([-0.00899099, -0.00881597,  0.00228988, ...,  0.00162625,
       -0.02012115,  0.00058665], dtype=float32)

In [34]:
query = (
    Query("(@tag:{ openai })=>[KNN 2 @vector $vec as score]")
     .sort_by("score")
     .return_fields("content", "nivo3", "nivo3A", "tag", "score")
     .paging(0, 2)
     .dialect(2)
)

query_params = {"vec": q_em.tobytes()}
temp_json = r.ft(INDEX_NAME).search(query, query_params).docs



In [51]:
for doc in temp_json:
    print("Taxonomy:", doc['content'], "\nNivo3:", doc['nivo3'], "\nNivo3A:", doc['nivo3A'], "\n\n")

Taxonomy: Kaj je Triglav komplet?  
Nivo3: Triglav komplet  
Nivo3A: KATEGORIZACIJA - NE MOREM SPREMENITI  


Taxonomy: Kako lahko sklenem zavarovanje?  
Nivo3: Naložbena in investicijska zavarovanja  
Nivo3A: Investicijsko zavarovanje fleks prek spleta - i.fleks  


